In [6]:
from os import truncate

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, trim, to_date, sum as spark_sum, max as spark_max, min as spark_min

spark = SparkSession.builder.appName("OnlineBankingAnalysis").getOrCreate()

loan_df = spark.read.option("header", "true").csv("/content/loan.csv", inferSchema=True)
credit_df = spark.read.option("header", "true").csv("/content/credit card.csv", inferSchema=True)
txn_df = spark.read.option("header", "true").csv("/content/txn.csv", inferSchema=True)

loan_df = loan_df.selectExpr([f"`{c}` as `{c.strip().replace(' ', '_')}`" for c in loan_df.columns])
credit_df = credit_df.selectExpr([f"`{c}` as `{c.strip().replace(' ', '_')}`" for c in credit_df.columns])
txn_df = txn_df.selectExpr([f"`{c}` as `{c.strip().replace(' ', '_')}`" for c in txn_df.columns])

loan_df = loan_df.withColumn("Loan_Amount_CLEAN", regexp_replace(trim(col("Loan_Amount")), ",", "").cast("double"))

txn_df = txn_df.withColumn("WITHDRAWAL_AMT_CLEAN", regexp_replace(trim(col("WITHDRAWAL_AMT")), ",", "").cast("double"))
txn_df = txn_df.withColumn("DEPOSIT_AMT_CLEAN", regexp_replace(trim(col("DEPOSIT_AMT")), ",", "").cast("double"))
txn_df = txn_df.withColumn("BALANCE_CLEAN", regexp_replace(trim(col("BALANCE_AMT")), ",", "").cast("double"))
txn_df = txn_df.withColumn("DATE", to_date("VALUE_DATE", "dd-MM-yyyy"))

# ---------------- LOAN DATA ----------------

print("1.Number of loans in each category")
loan_df.groupBy("Loan_Category").count().show()



1.Number of loans in each category
+------------------+-----+
|     Loan_Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



In [7]:

print("2.People who took loan > 1 lakh")
filtered_loan_1 = loan_df.filter(col("Loan_Amount_CLEAN") > 100000)
print("Count:", filtered_loan_1.count())
filtered_loan_1.show(truncate=False )

2.People who took loan > 1 lakh
Count: 450
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|Customer_ID|Age|Gender|Occupation         |Marital_Status|Family_Size|Income|Expenditure|Use_Frequency|Loan_Category   |Loan_Amount|Overdue|Debt_Record|Returned_Cheque|Dishonour_of_Bill|Loan_Amount_CLEAN|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|IB14001    |30 |MALE  |BANK MANAGER       |SINGLE        |4          |50000 |22199      |6            |HOUSING         | 10,00,000 |5      |42,898     |6              |9                |1000000.0        |
|IB14018    |29 |MALE  |TEACHER            |MARRIED       |5          |45767 |12787      |3            |GOLD LOAN       | 6,00,000  |

In [8]:

print("\n3. People with income > 60000")
filtered_income = loan_df.filter(col("Income") > 60000)
print("Count:", filtered_income.count())
filtered_income.show()



3. People with income > 60000
Count: 198
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|Customer_ID|Age|Gender|         Occupation|Marital_Status|Family_Size|Income|Expenditure|Use_Frequency|     Loan_Category|Loan_Amount|Overdue|Debt_Record|Returned_Cheque|Dishonour_of_Bill|Loan_Amount_CLEAN|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|    IB14032| 24|  MALE|       DATA ANALYST|        SINGLE|          4| 60111|      28999|            6|        AUTOMOBILE|     35,232|      5|     33,333|              1|                2|          35232.0|
|    IB14042| 25|FEMALE|             DOCTOR|        SINGLE|          4| 60111|      27111|            5|        TRAVELLING| 12

In [9]:

print("\n4. People with 2+ returned cheques and income < 50000")
filtered_returned_1 = loan_df.filter((col("Returned_Cheque") >= 2) & (col("Income") < 50000))
print("Count:", filtered_returned_1.count())
filtered_returned_1.show()



4. People with 2+ returned cheques and income < 50000
Count: 137
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+---------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|Customer_ID|Age|Gender|       Occupation|Marital_Status|Family_Size|Income|Expenditure|Use_Frequency|  Loan_Category|Loan_Amount|Overdue|Debt_Record|Returned_Cheque|Dishonour_of_Bill|Loan_Amount_CLEAN|
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+---------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|    IB14025| 39|FEMALE|          TEACHER|       MARRIED|          6| 46619|      18675|            4|        HOUSING| 12,09,867 |      8|     29,999|              6|                8|        1209867.0|
|    IB14027| 51|  MALE|   SYSTEM MANAGER|       MARRIED|          3| 49999|      19111|            5|    RESTAURANTS|    

In [10]:

print("\n5. People with 2+ returned cheques and Single")
filtered_returned_2 = loan_df.filter((col("Returned_Cheque") >= 2) & (col("Marital_Status") == "SINGLE"))
print("Count:", filtered_returned_2.count())
filtered_returned_2.show()



5. People with 2+ returned cheques and Single
Count: 111
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|Customer_ID|Age|Gender|       Occupation|Marital_Status|Family_Size|Income|Expenditure|Use_Frequency|     Loan_Category|Loan_Amount|Overdue|Debt_Record|Returned_Cheque|Dishonour_of_Bill|Loan_Amount_CLEAN|
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|    IB14001| 30|  MALE|     BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|     42,898|              6|                9|        1000000.0|
|    IB14012| 30|FEMALE|          DENTIST|        SINGLE|          3| 58450|      27675|            5|        TRAVELLI

In [11]:

print("\n6. People with monthly expenses > 50000")
filtered_expense = loan_df.filter(col("Expenditure") > 50000)
print("Count:", filtered_expense.count())
filtered_expense.show()




6. People with monthly expenses > 50000
Count: 6
+-----------+---+------+---------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|Customer_ID|Age|Gender|     Occupation|Marital_Status|Family_Size|Income|Expenditure|Use_Frequency|Loan_Category|Loan_Amount|Overdue|Debt_Record|Returned_Cheque|Dishonour_of_Bill|Loan_Amount_CLEAN|
+-----------+---+------+---------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+-----------+---------------+-----------------+-----------------+
|    IB14158| 54|  MALE|AIRPORT OFFICER|       MARRIED|          6| 80000|      62541|            2|   AUTOMOBILE| 20,45,789 |      1|     16,599|              2|                3|        2045789.0|
|    IB14176| 54|  MALE|AIRPORT OFFICER|       MARRIED|          6| 80000|      62541|            2|      HOUSING| 20,45,789 |      1|     16,599|        

In [12]:

# ---------------- CREDIT CARD DATA ----------------

print("7. Credit card users in Spain")
spain_credit_users = credit_df.filter(col("Geography") == "Spain")
print("Count:", spain_credit_users.count())
spain_credit_users.show(truncate=False)



7. Credit card users in Spain
Count: 2477
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|Surname  |CreditScore|Geography|Gender|Age|Tenure|Balance  |NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|2        |15647311  |Hill     |608        |Spain    |Female|41 |1     |83807.86 |1            |1             |112542.58      |0     |
|5        |15737888  |Mitchell |850        |Spain    |Female|43 |2     |125510.82|1            |1             |79084.1        |0     |
|6        |15574012  |Chu      |645        |Spain    |Male  |44 |8     |113755.78|2            |0             |149756.71      |1     |
|12       |15737173  |Andrews  |497        |Spain    |Male  |24 |3     |0.0      |2            |0             |76390.01       |0     |
|15       |15

In [13]:

# ---------------- TRANSACTION DATA ----------------

print("8.Maximum withdrawal amount")
txn_df.select(spark_max("WITHDRAWAL_AMT_CLEAN").alias("Max_Withdrawal")).show()



8.Maximum withdrawal amount
+--------------+
|Max_Withdrawal|
+--------------+
| 4.594475464E8|
+--------------+



In [14]:

print("9.Minimum withdrawal amount")
txn_df.select(spark_min("WITHDRAWAL_AMT_CLEAN").alias("Min_Withdrawal")).show()



9.Minimum withdrawal amount
+--------------+
|Min_Withdrawal|
+--------------+
|          0.01|
+--------------+



In [15]:
print("10.Maximum deposit amount")
txn_df.select(spark_max("DEPOSIT_AMT_CLEAN").alias("Max_Deposit")).show()


10.Maximum deposit amount
+-----------+
|Max_Deposit|
+-----------+
|    5.448E8|
+-----------+



In [16]:

print("11.Minimum deposit amount")
txn_df.select(spark_min("DEPOSIT_AMT_CLEAN").alias("Min_Deposit")).show()


11.Minimum deposit amount
+-----------+
|Min_Deposit|
+-----------+
|       0.01|
+-----------+



In [17]:

print("12.Total balance in each account")
txn_df.groupBy("Account_No").agg(spark_sum("BALANCE_CLEAN").alias("Total_Balance")).show(truncate = False)



12.Total balance in each account
+-------------+----------------------+
|Account_No   |Total_Balance         |
+-------------+----------------------+
|409000438611'|-2.4948657706833955E12|
|1196711'     |-1.60476498101275E13  |
|1196428'     |-8.1418498130721E13   |
|409000493210'|-3.2758495213209575E12|
|409000611074'|1.615533622E9         |
|409000425051'|-3.7721184116499877E9 |
|409000405747'|-2.4310804706700016E10|
|409000362497'|-5.2860004792808E13   |
|409000493201'|1.0420831829499985E9  |
|409000438620'|-7.122918679513586E12 |
+-------------+----------------------+



In [18]:
print("\n13. Customers with withdrawal > 1 lakh")
high_withdrawal = txn_df.filter(col("WITHDRAWAL_AMT_CLEAN") > 100000).select("Account_No", "WITHDRAWAL_AMT_CLEAN").distinct()
print("Count:", high_withdrawal.count())
high_withdrawal.show(truncate=False)


13. Customers with withdrawal > 1 lakh
Count: 10058
+-------------+--------------------+
|Account_No   |WITHDRAWAL_AMT_CLEAN|
+-------------+--------------------+
|409000611074'|274600.0            |
|409000493201'|1500000.0           |
|409000493201'|199604.27           |
|409000438620'|186604.0            |
|409000438620'|3.6675558E7         |
|1196711'     |7530283.0           |
|1196428'     |812361.0            |
|1196428'     |6348768.0           |
|1196428'     |3043151.63          |
|409000362497'|576954.0            |
|409000362497'|3423962.0           |
|409000362497'|3.144482503E7       |
|1196428'     |4441827.47          |
|409000611074'|145450.0            |
|409000493201'|119401.28           |
|1196711'     |628945.0            |
|1196428'     |289670.04           |
|409000362497'|3.483281361E7       |
|409000362497'|4.289763641E7       |
|409000362497'|2.678162613E7       |
+-------------+--------------------+
only showing top 20 rows

